# Import libraries & Setup dataframe

In [1]:
import pandas as pd

In [8]:
data_file = 'discover_2024-03-26.xlsx'
file_path = 'data/' + data_file
df = pd.read_excel(file_path, sheet_name='data')

FileNotFoundError: [Errno 2] No such file or directory: 'data/discover_2024-03-26.xlsx'

In [9]:
# keep raw data for later
df_raw = df.copy()

NameError: name 'df' is not defined

# Preprocessing of the data

In [ ]:
# transform columns to lower columns

columns = [col.lower() for col in df.columns]
df.columns = columns

# show results
df.head()

# Getting to know the data

In [111]:
df.describe()

,date,published_at,daily_likes,daily_dislikes,word_count,video_play,clickouts,discover_clicks,discover_impressions
count,132615,42111,3.360900e+04,2.728200e+04,41624.000000,131809.000000,131809.000000,1.318090e+05,1.318090e+05
mean,2023-08-11 08:35:05.680353024,2023-11-03 22:30:28.595853824,3.585587e+05,2.693901e+05,665.405655,1440.483730,24.940884,2.730320e+03,3.139446e+04
min,2023-01-01 00:00:00,2019-02-18 00:00:00,-8.400000e+06,-5.900000e+06,100.000000,0.000000,0.000000,0.000000e+00,5.000000e+01
25%,2023-05-05 00:00:00,2023-09-06 00:00:00,0.000000e+00,0.000000e+00,415.000000,7.000000,0.000000,2.500000e+01,2.810000e+02
50%,2023-08-03 00:00:00,2023-12-13 00:00:00,0.000000e+00,0.000000e+00,528.000000,54.000000,0.000000,1.480000e+02,2.142000e+03
75%,2023-11-22 00:00:00,2024-01-29 00:00:00,1.000000e+05,1.000000e+05,689.000000,379.000000,3.000000,9.460000e+02,1.227600e+04
max,2024-03-22 00:00:00,2024-03-21 00:00:00,2.568000e+08,2.629000e+08,5306.000000,703622.000000,18356.000000,1.053606e+06,1.088435e+07
std,NaN,NaN,3.878383e+06,4.272602e+06,495.550724,8960.408441,188.727723,1.542447e+04,1.588114e+05


In [121]:
df.page_efahrer_id.unique().shape

(6891,)

## Getting a better understanding of the features

Is the ID in PAGE_EFAHRER_ID the same as the ID in PAGE_NAME?

In [172]:
# Transform data type for the containment check
df['page_efahrer_id'] = df['page_efahrer_id'].astype('string')
df['page_name'] = df['page_name'].astype('string')

# Function to check whether Page ID is part of page name
def check_containment(row):
    return row['page_efahrer_id'] in row['page_name']

df['containment_check'] = df.apply(check_containment, axis=1)

# Check whether results consist only True values
df['containment_check'].unique()

# Yes, the IDs are always the same

# Drop row and transform data type back
df.drop('containment_check', axis=1, inplace=True)
df['page_efahrer_id'] = df['page_efahrer_id'].astype('int')

array([ True])

# Missing data

In [178]:
df.isna().sum()

page_efahrer_id                    0
date                               0
published_at                   90735
publish_date_equal_to_date         0
page_canonical_url                 0
page_name                          0
classification_product           655
classification_type              655
title                              0
page_author                        0
daily_likes                    99223
daily_dislikes                105555
word_count                     91207
video_play                       776
impressions                      776
discover_clicks                  776
discover_impressions             776
dtype: int64

Publish date "published_at" can be imputed by setting publish date to first date of occurrence in the data set.
Classification Type "classification_type" can be imputed by extracting it from URL.


# TBD IMPUTING OF MISSING DATA

# Scrape data (tittles, actual date)

In [1]:
# unique ids
scraping = df["page_efahrer_id", "page_canonical_url"].drop_duplicates(subset = ["page_efahrer_id"])
scraping
scraping = scraping.to_frame()

NameError: name 'df' is not defined

In [ ]:
# creating the dummy columns that will be populated with the scraped data
scraping['H1'] = 'dummy'
scraping['last_update_date'] = 'today'
scraping['abstract'] = 'null'

scraping = scraping.set_index('page_efahrer_id')

scraping
## Do actual scraping

i=0
for row_idx in scraping.index:
    scraping.loc[row_idx, 'H1'] = ##the header
    scraping.loc[row_idx, 'last_update_date'] = ##the last update time element
    scraping.loc[row_idx, 'abstract'] = ##find a clever way to extract the abstract?
    i+=1
    if i==10: 
        scraping.to_csv('../data/temp_scraped.csv')
        i=0

# To be sorted/ organized

In [130]:
df_pub = df.query("publish_date_equal_to_date == 'Y'")

In [131]:
df.page_efahrer_id.unique()

<StringArray>
['1010629',  '108566', '1010799', '1010717',  '109654',  '109702',  '105640',
 '1010317',  '107406', '1010297',
 ...
 '1018760', '1018763', '1018739', '1018749', '1018744', '1018643', '1018684',
 '1018764', '1018672', '1018511']
Length: 6891, dtype: string

In [132]:
df_pub.page_efahrer_id.unique()

<StringArray>
['1010717', '1010636', '1010720', '1010781', '1010746', '1010719',  '107838',
 '1010317',  '107766', '1010865',
 ...
 '1016553', '1016560', '1016559', '1016555', '1016552', '1016350', '1016567',
 '1011711', '1016568', '1016543']
Length: 4632, dtype: string

In [133]:
print(df.daily_likes.isna().sum())
print(df.daily_likes.notna().sum())

99006
33609


In [135]:
df.groupby("page_efahrer_id").daily_likes.head()


0              NaN
1              NaN
2              NaN
3         800000.0
4              NaN
            ...   
132597    800000.0
132601         0.0
132607         NaN
132612         NaN
132614         0.0
Name: daily_likes, Length: 32158, dtype: float64

In [138]:
df.head()

,page_efahrer_id,date,published_at,publish_date_equal_to_date,page_canonical_url,page_name,classification_product,classification_type,title,page_author,daily_likes,daily_dislikes,word_count,video_play,clickouts,discover_clicks,discover_impressions,containment_check
0,1010629,2023-01-01,NaT,N,https://efahrer.chip.de/news/unwirksame-preise...,efa-1010629 | StromerhÃ¶hungs-Brief gar nicht ...,Energie,News,"Unwirksame PreiserhÃ¶hung: Wer hier Kunde ist,...",Vanessa Finkler,NaN,NaN,NaN,0.0,0.0,1385.0,12410.0,True
1,108566,2023-01-01,NaT,N,https://efahrer.chip.de/news/laedt-super-schne...,efa-108566 | Stromausfall mitten in Deutschlan...,Solargenerator,News,LÃ¤dt super schnell: Wir haben den Top-Solarge...,Sebastian Barsch,NaN,NaN,NaN,130.0,2.0,143.0,8083.0,True
2,1010799,2023-01-01,NaT,N,https://efahrer.chip.de/news/gedrosselter-stro...,efa-1010799 | 10 Stunden und lÃ¤nger - Netzage...,E-Auto,News,Gedrosselter Strom fÃ¼r E-Autos und WÃ¤rmepump...,FOCUS Online,NaN,NaN,NaN,491.0,2.0,803.0,13542.0,True
3,1010717,2023-01-01,2023-01-01,Y,https://efahrer.chip.de/news/heizung-faellt-im...,efa-1010717 | Dacia Spring hat massives Heizpr...,E-Auto,News,Heizung fÃ¤llt im Dacia Spring aus: Wie Fahrer...,Tobias Stahl,800000.0,200000.0,914.0,140.0,1.0,266.0,18289.0,True
4,109654,2023-01-01,NaT,N,https://efahrer.chip.de/news/bis-zu-12-stunden...,efa-109654 | 12 Stunden Strom-Ausfall pro Tag:...,Energie,News,Ð”Ð¾ 12 Ñ‡Ð°ÑÐ¾Ð² Ð±ÐµÐ· ÑÐ»ÐµÐºÑ‚Ñ€Ð¸Ñ‡ÐµÑ...,Aslan Berse,NaN,NaN,NaN,133.0,0.0,312.0,6018.0,True


In [142]:
import matplotlib as plt
import seaborn as sns

In [149]:
daily_likes = df.daily_likes.
daily_likes
#sns.histplot(df.daily_likes.unique())

AttributeError: 'Series' object has no attribute 'daily_likes'

In [158]:
non_null_likes = df['daily_likes'].dropna()
non_null_likes.unique().min()
#sns.histplot(non_null_likes)

-8400000.0

In [140]:
df.page_efahrer_id.unique().shape

(6891,)

In [ ]:
df.drop_duplicates("page_efahrer_id", "date", "DISCOVER_IMPRESSIONS")

In [101]:
df.groupby("page_efahrer_id").count()

,DATE,PUBLISHED_AT,PUBLISH_DATE_EQUAL_TO_DATE,PAGE_CANONICAL_URL,PAGE_NAME,CLASSIFICATION_PRODUCT,CLASSIFICATION_TYPE,TITLE,PAGE_AUTHOR,DAILY_LIKES,DAILY_DISLIKES,WORD_COUNT,VIDEO_PLAY,CLICKOUTS,DISCOVER_CLICKS,DISCOVER_IMPRESSIONS,containment_check
PAGE_EFAHRER_ID,,,,,,,,,,,,,,,,,
1010012,8,0,8,8,8,8,8,8,8,0,0,0,8,8,8,8,8
1010022,20,4,20,20,20,20,20,20,20,4,4,4,20,20,20,20,20
1010045,85,23,85,85,85,85,85,85,85,23,23,23,85,85,85,85,85
1010048,156,62,156,156,156,156,156,156,156,62,62,62,156,156,156,156,156
1010057,1,0,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109925,34,3,34,34,34,34,34,34,34,2,0,3,34,34,34,34,34
109934,102,11,102,102,102,102,102,102,102,11,11,11,102,102,102,102,102
109968,55,17,55,55,55,55,55,55,55,17,17,17,55,55,55,55,55


In [108]:
df[["PAGE_EFAHRER_ID", "PAGE_NAME"]].drop_duplicates().groupby("PAGE_NAME").count().max()

PAGE_EFAHRER_ID    1
dtype: int64